#  Gruppuppgift Machine Learning
- Machine Learning - Kunskapskontroll 1

## Läser in data till Kunskapskontroll 1

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

Data_path = r"G:\Min enhet\Mina Dokument\Utbildning\BI25\Machine Learning\Kunskapskontroll_gruppuppgift\car_price_dataset.csv"

df_carprices_orig = pd.read_csv(Data_path, sep=';', encoding='cp1252')       # Hittade att det blev en kolumn, separerar csv-filens data med ;
df_carprices = df_carprices_orig.copy()                                 # Sparar en kopia av originaldataframen
    

### Dataset overview

In [9]:
df_carprices.info()     # Information om datatyper och icke-null värden i varje kolumn
# Inga nullvärden
# 10 000 rader
# 10 koumner

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Brand         10000 non-null  object 
 1   Model         10000 non-null  object 
 2   Year          10000 non-null  int64  
 3   Engine_Size   10000 non-null  float64
 4   Fuel_Type     10000 non-null  object 
 5   Transmission  10000 non-null  object 
 6   Mileage       10000 non-null  int64  
 7   Doors         10000 non-null  int64  
 8   Owner_Count   10000 non-null  int64  
 9   Price         10000 non-null  int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 781.4+ KB


In [10]:
df_carprices.head(10)    # 10 första raderna i min dataframe med data från car_price_dataset.csv för att utforska datan

# Kategoriska kolumner: Brand, Model, Fuel_Type, Transmission
# Numeriska kolumner: Year, Mileage, Doors, Owner_Count, Engine_Size

,Brand,Model,Year,Engine_Size,Fuel_Type,Transmission,Mileage,Doors,Owner_Count,Price
0,Kia,Rio,2020,4.2,Diesel,Manual,289944,3,5,8501
1,Chevrolet,Malibu,2012,2.0,Hybrid,Automatic,5356,2,3,12092
2,Mercedes,GLA,2020,4.2,Diesel,Automatic,231440,4,2,11171
3,Audi,Q5,2023,2.0,Electric,Manual,160971,2,1,11780
4,Volkswagen,Golf,2003,2.6,Hybrid,Semi-Automatic,286618,3,3,2867
5,Toyota,Camry,2007,2.7,Petrol,Automatic,157889,4,4,7242
6,Honda,Civic,2010,3.4,Electric,Automatic,139584,3,1,11208
7,Kia,Sportage,2001,4.7,Electric,Semi-Automatic,157495,2,2,7950
8,Kia,Sportage,2014,2.6,Hybrid,Manual,98700,3,4,9926
9,Toyota,RAV4,2005,3.1,Petrol,Manual,107724,2,5,6545


In [11]:
df_carprices.describe() # Statistisk sammanfattning av numeriska kolumner

# Möjliga Outliers: Mileage: min 25 och max 299 947, mean 149 239
# Diskreta variabler med få nivåer: Doors, Owner_Count
# Kategoriska variabler med många nivåer: Brand, Model
# Beroende variabel: Price
# Oberoende variabler: Year, Mileage, Doors, Owner_Count, Engine_Size, Brand, Model, Fuel_Type, Transmission

,Year,Engine_Size,Mileage,Doors,Owner_Count,Price
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000
mean,2011.543700,3.000560,149239.111800,3.497100,2.991100,8852.96440
std,6.897699,1.149324,86322.348957,1.110097,1.422682,3112.59681
min,2000.000000,1.000000,25.000000,2.000000,1.000000,2000.00000
25%,2006.000000,2.000000,74649.250000,3.000000,2.000000,6646.00000
50%,2012.000000,3.000000,149587.000000,3.000000,3.000000,8858.50000
75%,2017.000000,4.000000,223577.500000,4.000000,4.000000,11086.50000
max,2023.000000,5.000000,299947.000000,5.000000,5.000000,18301.00000


In [12]:
X = df_carprices.drop(columns=["Price"])        # Sätter alla kolumner utom målvariabeln Price i X
y = df_carprices["Price"]                       # Sätter kolumnen Price i målvariabeln y (target)

### Uppdelning av data i träningsdata och testdata

In [13]:
from sklearn.model_selection import train_test_split

X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=40)                 # delar upp data i 80% träning/validering och 20% test
X_train, X_val, y_train, y_val  = train_test_split(X_train_full, y_train_full, test_size=0.25, random_state=40)     # delar upp träingsdata i 75% träning (0.75 * 0.80 = 0.60 = 60% av hela datasetet) och 25% validering (0.25 * 0.80 = 0.20 = 20% av hela datasetet)

print("Train:", X_train.shape, y_train.shape)
print("Val:  ", X_val.shape, y_val.shape)
print("Test: ", X_test.shape, y_test.shape)

Train: (6000, 9) (6000,)
Val:   (2000, 9) (2000,)
Test:  (2000, 9) (2000,)


### Preprocessing av data

#### Förvandlar till numeriska kolumner

In [14]:
from sklearn.compose import ColumnTransformer                   
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Delar upp kolumner i kategoriska och numeriska
categorical_cols = ["Brand", "Model", "Fuel_Type", "Transmission"]              # Delar upp kolumnerna i icke siffor              
numeric_cols = [c for c in X_train.columns if c not in categorical_cols]        # Och kolumner med siffror

# Skapar en ColumnTransformer, en för linear modeller
ct_linear = ColumnTransformer(transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore", drop="first"), categorical_cols),        # För Categorical/kategori (text): gör om till siffror med One-Hot (0/1-kolumner) drop="first" för att undvika dummy-variabel-fälla, fungerar både till linear och tree modeller
        ("numeric", "passthrough", numeric_cols),])                                                     # För numeriska kolumner: låt dem vara som de är ("passthrough")

# Skapar en ColumnTransformer, en för tree modeller
ct_tree = ColumnTransformer(transformers=[  
        ("categorical", OneHotEncoder(handle_unknown="ignore"), categorical_cols),                      # ColumnTransformer för tree modeller, ingen drop="first" här   
        ("numeric", "passthrough", numeric_cols),])                                                     # För numeriska kolumner: låt dem vara som de är ("passthrough")

# Skapar en ColumnTransformer med skalning för linear modeller
ct_scaled = ColumnTransformer(transformers=[
        ("categorical", OneHotEncoder(handle_unknown="ignore", drop="first"), categorical_cols),        # För Categorical/kategori (text): gör om till siffror med One-Hot (0/1-kolumner) drop="first" för att undvika dummy-variabel-fälla
        ("numeric", StandardScaler(), numeric_cols)])                                                   # För numeriska kolumner: skala med StandardScaler (medelvärde 0, standardavvikelse 1)


### EDA

In [ ]:
# Kollar efter dubbletter i träningsdata
# 0 dubbletter
print("Duplicerade rader X_train:", X_train.duplicated().sum())
print("Duplicerade rader y_train:", y_train.duplicated().sum())

# Saknade värden (du har inga, men bra att visa)
print("\nMissing values (train):")
print(train_df.isna().sum().sort_values(ascending=False).head(10))